In [62]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import glob
!pip install openpyxl

day1_folder = "/content/drive/My Drive/11 april/"

file_paths = glob.glob(day1_folder + "*.csv")

dataframes = [pd.read_csv(file, sep=';') for file in file_paths]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
keyfile_general = pd.read_csv("Keyfile_csv.csv")
keyfile_102 = pd.read_excel("keyfile school 42 class 102.xlsx")
keyfile_specific = keyfile_102

file_to_subject = {}

for file in file_paths:
    df_temp = pd.read_csv(file, sep=';')
    tag_id = df_temp['TagId'].iloc[0]  # This is the TrackLabID
    df = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    try:
        subject_id = keyfile_specific.loc[keyfile_specific['tagnr'] == tagnr, 'subject ID'].values[0]
    except:
        print("Tag not found:" + tag_id)
        subject_id = None  # Assign None if subject ID not found

    # Store TrackLabID (tag_id), Tagnumber, and subject ID in the dictionary
    file_to_subject[tag_id] = {'Tagnumber': tagnr, 'subject ID': subject_id}
#print(file_to_subject)


Tag not found:0x24025F44CDE1
Tag not found:0x24046130BA41
Tag not found:0x24046131F1EE


In [64]:
def fix_coordinates(value):

    #fixes issues of values like 940721624896047
    while abs(value) > 10:
        value /= 10  # moves the decimal one to the left

    return value

#just a sanity check, will remove in final version
for df in dataframes:
    for col in ['X', 'Y', 'Z']:
        df[col] = df[col].apply(fix_coordinates)
    print(df['X'].max())
    print(df['X'].min())

9.40721624896047
0.453501952361497
9.92450105308614
-9.96295390741081
9.60256903603166
-3.32349787729596
9.85197566046511
-8.86466138960537
9.98594484804531
-9.87490160488661
9.86704157508411
-6.15536133797028
9.36340432184005
-9.75192651096805
9.95345396150243
-9.90416330663674
9.93403314592184
-8.84926791189599
9.9188945205304
-9.97887611646083
9.47492035396462
-6.20691810279558
9.52685260676476
-3.95312494589235
9.82162676087264
-8.29262587240004
9.22202465435961
-9.28494079485871
9.80214529547855
-9.94566500674398
9.33284780905031
-9.77106388810018


In [65]:
#don't actually use the averages, could be used for plotting though (I removed those cells from the notebook because we don't actually need it, could add back if you want)
object_averages = {}
teacher_tag = '0x24046130BA41'
for index, df_ in enumerate(dataframes):
    tag_id = df_['TagId'].iloc[0]
    df = pd.read_csv("Keyfile_csv.csv", sep=';')


    if tag_id == teacher_tag:
        tagnr = '35'
        subject_id = "Teacher"

    elif tag_id in df['TrackLabID'].values:
        tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

        try:
            subject_id = keyfile_specific.loc[keyfile_specific['tagnr'] == tagnr, 'subject ID'].values[0]
        except (KeyError, IndexError):
            print(f"Tagnr {tagnr} not found in keyfile_specific")
            subject_id = None


    # Calculate averages only if subject_id is not None
    if subject_id is not None:
        avg_x, avg_y, avg_z = df_[['X', 'Y', 'Z']].mean()
        object_averages[f"Person_{subject_id} with Tagnr {tagnr}"] = (avg_x, avg_y, avg_z)

avg_df = pd.DataFrame.from_dict(object_averages, orient="index", columns=["X", "Y", "Z"])
print(avg_df)

Tagnr 29 not found in keyfile_specific
Tagnr 33 not found in keyfile_specific
                                     X         Y         Z
Person_7.0 with Tagnr 7       4.576558  3.536418  0.464580
Person_10.0 with Tagnr 11     4.244901  1.513698  0.509476
Person_8.0 with Tagnr 9       3.341943  3.126551  0.388473
Person_Teacher with Tagnr 35  2.690934  4.019353  0.577932
Person_14.0 with Tagnr 18     4.085606  2.736085  0.445339
Person_15.0 with Tagnr 17     2.939757  2.210103  0.385922
Person_9.0 with Tagnr 10      3.261954  3.740230  0.450669
Person_6.0 with Tagnr 6       2.813791  1.765423  0.328490
Person_3.0 with Tagnr 2       3.932704  2.785945  0.447917
Person_4.0 with Tagnr 3       4.100720  3.587963  0.447006
Person_1.0 with Tagnr 1       2.195395  3.485029  0.432202
Person_11.0 with Tagnr 13     1.016909  3.761704  0.554393
Person_5.0 with Tagnr 5       4.262624  4.356669  0.502038
Person_13.0 with Tagnr 15     1.937024  2.766699  0.416927


In [66]:
fivemin_object_averages = {}
print(dataframes[0].columns)
for index, df_ in enumerate(dataframes):
    tag_id = df_['TagId'].iloc[0]
    #print(tag_id)
    df = pd.read_csv("Keyfile_csv.csv", sep=';')


    tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]
    # Calculate average for the first 300 rows (approximately 5 minutes)
    avg_x, avg_y, avg_z = df_[['X', 'Y', 'Z']].head(300).mean()
    fivemin_object_averages[f"{tagnr}"] = (avg_x, avg_y, avg_z)

#Note I don't see much value in the Z-coordinate, but have included it for now

avg_df = pd.DataFrame.from_dict(fivemin_object_averages, orient="index", columns=["X", "Y", "Z"])

print(avg_df)

Index(['TimeStamp', 'X', 'Y', 'Z', 'TagId'], dtype='object')
           X         Y         Z
7   4.463590  4.014012  0.433734
11  4.134193  1.926773  0.565215
29  1.203719  1.564920  0.489788
9   3.066142  3.277840  0.375702
35  3.674897  5.099890  0.514209
18  3.321364  1.230519  0.304078
17  3.007175  2.026214  0.261815
10  3.565181  4.469656  0.505362
6   5.594847  3.753852  0.648147
2   4.986470  3.363611  0.429127
3   3.393116  3.277142  0.330418
1   1.577541  3.963902  0.458141
13  1.012145  4.052341  0.499792
5   4.266886  5.432391  0.484703
33  4.128696  3.271979  0.391307
15  1.631343  2.946449  0.472459


In [67]:
#this initially was for baseline distance purposes, decided using avg first 5 minutes might be better though due to trackers potentially not being turned on in usual positions (e.g. turning it on whilst picking it up from teachers desk)
object_start = {}

for index, df_ in enumerate(dataframes):
    tag_id = df_['TagId'].iloc[0]
    df = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]
    # Get the first coordinates for X, Y, and Z
    start_x = df_['X'].iloc[0]
    start_y = df_['Y'].iloc[0]
    start_z = df_['Z'].iloc[0]
    object_start[f"{tagnr}"] = (start_x, start_y, start_z)

start_positions_df = pd.DataFrame.from_dict(object_start, orient="index", columns=["X", "Y", "Z"])

print(start_positions_df)

           X         Y         Z
7   4.378336  3.858561  0.519511
11  4.137252  1.136436  0.547842
29  1.107881  0.706208  0.743003
9   5.180991  3.668428  0.403377
35  2.322620  5.061799  0.677834
18  3.228317  1.148220  0.224174
17  3.007175  2.026214  0.504837
10  3.570728  4.738615  0.631311
6   5.477901  3.812096  0.405335
2   5.304984  3.610548  0.408014
3   4.663750  4.402218  0.126461
1   1.834673  4.049643  0.235754
13  1.232346  4.027922  0.643666
5   4.257247  5.243555  0.548409
33  4.215339  3.362791  0.388413
15  1.594550  2.928901  0.427727


In [68]:
import numpy as np

def euclidean_distance(x1, y1, x2, y2):
  return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

target_tagnr = 35

# Find target coordinates using tagnr
target_coords = None
for tracker, (x, y, _) in fivemin_object_averages.items():
  if tracker == f"{target_tagnr}":
    target_coords = (x, y)
    break




target_x, target_y = target_coords
min_distance = float('inf')
closest_tracker = None

for tracker, (x, y, _) in fivemin_object_averages.items():
  if tracker != f"{target_tagnr}":  # Excludes teacher who would have distance 0
    distance = euclidean_distance(target_x, target_y, x, y)
    print(distance, tracker) #can be removed later
    if distance < min_distance:
      min_distance = distance
      closest_tracker = tracker

closest_distance_to_35 = min_distance
print(f"The closest tracker to the teacher is {closest_tracker} with a distance of {closest_distance_to_35}")

1.3420755508487303 7
3.2061846134616188 11
4.313088502235195 29
1.921053202552734 9
3.88548736150836 18
3.1453668801536976 17
0.6397123800646543 10
2.344787194276207 6
2.1759798891988034 2
1.8443995886034945 3
2.385240322275953 1
2.86139953178728 13
0.6789755505487487 5
1.8833988288822905 33
2.968740139758129 15
The closest tracker to the teacher is 10 with a distance of 0.6397123800646543


In [69]:
tracker_counts = {}

teacher_df = next((df for df in dataframes if df['TagId'].iloc[0] == teacher_tag), None)  # Find teacher

for df in dataframes:
    tag_id = df['TagId'].iloc[0]
    df_keyfile = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df_keyfile.loc[df_keyfile['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    # Skip teacher
    if tagnr == target_tagnr:
        continue

    # Skip unmatchable trackers
    if file_to_subject.get(tag_id) and file_to_subject[tag_id]['subject ID'] is None:
        continue

    # Initialise count for tracker
    tracker_counts[f"{tagnr}"] = 0

    # Get minimum length of both dataframes to avoid IndexError
    min_len = min(len(df), len(teacher_df))

    # Iterate through rows using the index
    for index in range(min_len):
        x = df.loc[index, 'X']
        y = df.loc[index, 'Y']

        # Get teacher's coordinates from the same row index
        target_x = teacher_df.loc[index, 'X']
        target_y = teacher_df.loc[index, 'Y']

        # Calculate distance to target
        distance = euclidean_distance(target_x, target_y, x, y)

        # Check if within desired distance
        if distance < closest_distance_to_35:
            tracker_counts[f"{tagnr}"] += 1

    # Sort tracker counts
sorted_tracker_counts = dict(sorted(tracker_counts.items(), key=lambda item: item[1], reverse=True))

# Print the sorted counts with subject IDs
for tracker, count in sorted_tracker_counts.items():
    subject_id = file_to_subject.get(df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0], {}).get('subject ID')
    print(f"Person {subject_id} with Tracker {tracker}: {count} times within distance of teacher")

Person 9.0 with Tracker 10: 2952 times within distance of teacher
Person 11.0 with Tracker 13: 2176 times within distance of teacher
Person 6.0 with Tracker 6: 2155 times within distance of teacher
Person 5.0 with Tracker 5: 1501 times within distance of teacher
Person 13.0 with Tracker 15: 1377 times within distance of teacher
Person 10.0 with Tracker 11: 1291 times within distance of teacher
Person 15.0 with Tracker 17: 1002 times within distance of teacher
Person 14.0 with Tracker 18: 797 times within distance of teacher
Person 3.0 with Tracker 2: 726 times within distance of teacher
Person 4.0 with Tracker 3: 483 times within distance of teacher
Person 8.0 with Tracker 9: 341 times within distance of teacher
Person 1.0 with Tracker 1: 319 times within distance of teacher
Person 7.0 with Tracker 7: 104 times within distance of teacher


In [71]:
half_distance = closest_distance_to_35 / 2
htracker_counts = {}

teacher_df = next((df for df in dataframes if df['TagId'].iloc[0] == teacher_tag), None)  # Find teacher


for df in dataframes:
    tag_id = df['TagId'].iloc[0]
    df_keyfile = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df_keyfile.loc[df_keyfile['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    # Skip teacher
    if tagnr == target_tagnr:
        continue

    # Skip unmatchable trackers
    if file_to_subject.get(tag_id) and file_to_subject[tag_id]['subject ID'] is None:
        continue

    # Initialise count for tracker
    htracker_counts[f"{tagnr}"] = 0

    # Get minimum length of both dataframes to avoid IndexError
    min_len = min(len(df), len(teacher_df))

    # Iterate through rows using the index
    for index in range(min_len):
        x = df.loc[index, 'X']
        y = df.loc[index, 'Y']

        # Get teacher's coordinates from the same row index
        target_x = teacher_df.loc[index, 'X']
        target_y = teacher_df.loc[index, 'Y']

        # Calculate distance to target
        distance = euclidean_distance(target_x, target_y, x, y)

        # Check if within desired distance
        if distance < half_distance:
            htracker_counts[f"{tagnr}"] += 1

    # Sort tracker counts
hsorted_tracker_counts = dict(sorted(htracker_counts.items(), key=lambda item: item[1], reverse=True))

# Print the sorted counts with subject IDs
for tracker, count in hsorted_tracker_counts.items():
    subject_id = file_to_subject.get(df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0], {}).get('subject ID')
    print(f"Person {subject_id} with Tracker {tracker}: {count} times within half distance of teacher")

Person 9.0 with Tracker 10: 842 times within half distance of teacher
Person 11.0 with Tracker 13: 513 times within half distance of teacher
Person 6.0 with Tracker 6: 458 times within half distance of teacher
Person 5.0 with Tracker 5: 314 times within half distance of teacher
Person 14.0 with Tracker 18: 276 times within half distance of teacher
Person 10.0 with Tracker 11: 267 times within half distance of teacher
Person 13.0 with Tracker 15: 214 times within half distance of teacher
Person 3.0 with Tracker 2: 131 times within half distance of teacher
Person 15.0 with Tracker 17: 125 times within half distance of teacher
Person 4.0 with Tracker 3: 90 times within half distance of teacher
Person 8.0 with Tracker 9: 65 times within half distance of teacher
Person 1.0 with Tracker 1: 43 times within half distance of teacher
Person 7.0 with Tracker 7: 37 times within half distance of teacher


In [73]:
quarter_distance = closest_distance_to_35/4

qtracker_counts = {}

teacher_df = next((df for df in dataframes if df['TagId'].iloc[0] == teacher_tag), None)  # Find teacher


for df in dataframes:
    tag_id = df['TagId'].iloc[0]
    df_keyfile = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df_keyfile.loc[df_keyfile['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    # Skip teacher
    if tagnr == target_tagnr:
        continue

    # Skip unmatchable trackers
    if file_to_subject.get(tag_id) and file_to_subject[tag_id]['subject ID'] is None:
        continue

    # Initialise count for tracker
    qtracker_counts[f"{tagnr}"] = 0

    # Get minimum length of both dataframes to avoid IndexError
    min_len = min(len(df), len(teacher_df))

    # Iterate through rows using the index
    for index in range(min_len):
        x = df.loc[index, 'X']
        y = df.loc[index, 'Y']

        # Get teacher's coordinates from the same row index
        target_x = teacher_df.loc[index, 'X']
        target_y = teacher_df.loc[index, 'Y']

        # Calculate distance to target
        distance = euclidean_distance(target_x, target_y, x, y)

        # Check if within desired distance
        if distance < quarter_distance:
            qtracker_counts[f"{tagnr}"] += 1

    # Sort tracker counts
qsorted_tracker_counts = dict(sorted(qtracker_counts.items(), key=lambda item: item[1], reverse=True))

# Print the sorted counts with subject IDs
for tracker, count in qsorted_tracker_counts.items():
    subject_id = file_to_subject.get(df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0], {}).get('subject ID')
    print(f"Person {subject_id} with Tracker {tracker}: {count} times within quarter distance of teacher")

Person 11.0 with Tracker 13: 166 times within quarter distance of teacher
Person 9.0 with Tracker 10: 156 times within quarter distance of teacher
Person 5.0 with Tracker 5: 89 times within quarter distance of teacher
Person 6.0 with Tracker 6: 81 times within quarter distance of teacher
Person 10.0 with Tracker 11: 71 times within quarter distance of teacher
Person 14.0 with Tracker 18: 64 times within quarter distance of teacher
Person 13.0 with Tracker 15: 27 times within quarter distance of teacher
Person 3.0 with Tracker 2: 25 times within quarter distance of teacher
Person 15.0 with Tracker 17: 22 times within quarter distance of teacher
Person 4.0 with Tracker 3: 21 times within quarter distance of teacher
Person 1.0 with Tracker 1: 14 times within quarter distance of teacher
Person 7.0 with Tracker 7: 13 times within quarter distance of teacher
Person 8.0 with Tracker 9: 9 times within quarter distance of teacher
